In [1]:
!pip install catboost


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Kütüphaneler

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import mean_squared_error , r2_score , confusion_matrix , accuracy_score , roc_auc_score , roc_curve , classification_report
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale , StandardScaler     # standardizasoyon işlemleri
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier



from warnings import filterwarnings
filterwarnings("ignore")     # önceki bölümlerde çalıştırdığımızda çıkan gereksiz uyarı mesajları çıkmasın diye

In [3]:
df = pd.read_csv("Veri ve Fotoğraflar\diabetes.csv")
df.head()


# Bağımlı değişkenimiz "Outcome". Yani burada kişi diyabet hastası mı değil mi bunu inceliyoruz. 1, hasta ; 0, hasta değil

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
y = df["Outcome"]

x = df.drop(["Outcome"] , axis=1)

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.30 , random_state=42)

# MODEL

In [5]:
catboost_model = CatBoostClassifier().fit(x_train , y_train)

Learning rate set to 0.0079
0:	learn: 0.6879744	total: 152ms	remaining: 2m 32s
1:	learn: 0.6843414	total: 155ms	remaining: 1m 17s
2:	learn: 0.6805760	total: 156ms	remaining: 52s
3:	learn: 0.6765782	total: 158ms	remaining: 39.3s
4:	learn: 0.6720829	total: 160ms	remaining: 31.8s
5:	learn: 0.6681617	total: 161ms	remaining: 26.7s
6:	learn: 0.6641912	total: 163ms	remaining: 23.1s
7:	learn: 0.6602881	total: 165ms	remaining: 20.5s
8:	learn: 0.6562626	total: 167ms	remaining: 18.4s
9:	learn: 0.6528483	total: 169ms	remaining: 16.7s
10:	learn: 0.6487823	total: 171ms	remaining: 15.3s
11:	learn: 0.6454980	total: 172ms	remaining: 14.2s
12:	learn: 0.6418953	total: 174ms	remaining: 13.2s
13:	learn: 0.6374431	total: 176ms	remaining: 12.4s
14:	learn: 0.6339676	total: 179ms	remaining: 11.8s
15:	learn: 0.6304252	total: 181ms	remaining: 11.2s
16:	learn: 0.6266998	total: 183ms	remaining: 10.6s
17:	learn: 0.6237660	total: 186ms	remaining: 10.1s
18:	learn: 0.6204394	total: 187ms	remaining: 9.68s
19:	learn: 0.

# TAHMİN

In [6]:
y_pred = catboost_model.predict(x_test)

accuracy_score(y_test , y_pred)

0.7402597402597403

# MODEL TUNING (DOĞRULAMA)

In [7]:
catboost = CatBoostClassifier()

In [8]:
catboost_params = {"iterations": [200,500,1000] , "learning_rate": [0.1 , 0.01 , 0.03] , "max_depth": [4,5,8]}

In [9]:
catboost_cv_model = GridSearchCV(catboost , catboost_params , cv=5 , n_jobs=-1 , verbose=2).fit(x_train , y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
0:	learn: 0.6863312	total: 4.04ms	remaining: 803ms
1:	learn: 0.6811157	total: 8.74ms	remaining: 866ms
2:	learn: 0.6752970	total: 12.2ms	remaining: 799ms
3:	learn: 0.6697432	total: 16.1ms	remaining: 788ms
4:	learn: 0.6641758	total: 19.6ms	remaining: 766ms
5:	learn: 0.6576656	total: 23ms	remaining: 743ms
6:	learn: 0.6521870	total: 26.3ms	remaining: 726ms
7:	learn: 0.6465564	total: 30.4ms	remaining: 730ms
8:	learn: 0.6407595	total: 33.8ms	remaining: 717ms
9:	learn: 0.6357725	total: 37.1ms	remaining: 704ms
10:	learn: 0.6299942	total: 40.9ms	remaining: 703ms
11:	learn: 0.6240480	total: 44.8ms	remaining: 702ms
12:	learn: 0.6196095	total: 48.2ms	remaining: 694ms
13:	learn: 0.6148715	total: 51.9ms	remaining: 689ms
14:	learn: 0.6092626	total: 55.7ms	remaining: 688ms
15:	learn: 0.6046764	total: 59.2ms	remaining: 681ms
16:	learn: 0.6012725	total: 62.7ms	remaining: 675ms
17:	learn: 0.5958121	total: 66.9ms	remaining: 676ms
18:	learn: 0.5

In [10]:
catboost_cv_model.best_params_

{'iterations': 200, 'learning_rate': 0.01, 'max_depth': 8}

In [11]:
# FİNAL MODELİMİZ

catboost_tuned = CatBoostClassifier(iterations=200 , learning_rate= 0.01 , max_depth=8).fit(x_train , y_train)

0:	learn: 0.6863312	total: 4.98ms	remaining: 991ms
1:	learn: 0.6811157	total: 9.85ms	remaining: 976ms
2:	learn: 0.6752970	total: 14.3ms	remaining: 941ms
3:	learn: 0.6697432	total: 17.9ms	remaining: 879ms
4:	learn: 0.6641758	total: 25.2ms	remaining: 982ms
5:	learn: 0.6576656	total: 30.3ms	remaining: 981ms
6:	learn: 0.6521870	total: 35.5ms	remaining: 978ms
7:	learn: 0.6465564	total: 41.5ms	remaining: 996ms
8:	learn: 0.6407595	total: 45.2ms	remaining: 960ms
9:	learn: 0.6357725	total: 50ms	remaining: 951ms
10:	learn: 0.6299942	total: 54.2ms	remaining: 931ms
11:	learn: 0.6240480	total: 58ms	remaining: 909ms
12:	learn: 0.6196095	total: 64.6ms	remaining: 929ms
13:	learn: 0.6148715	total: 68.3ms	remaining: 907ms
14:	learn: 0.6092626	total: 71.8ms	remaining: 886ms
15:	learn: 0.6046764	total: 75.1ms	remaining: 863ms
16:	learn: 0.6012725	total: 78.8ms	remaining: 848ms
17:	learn: 0.5958121	total: 82.5ms	remaining: 835ms
18:	learn: 0.5903243	total: 85.7ms	remaining: 816ms
19:	learn: 0.5866024	total

In [12]:
y_pred = catboost_tuned.predict(x_test)

accuracy_score(y_test , y_pred)

0.7532467532467533